In [1]:
from datetime import datetime
start_time = datetime.now()

# Libraries to install
# %pip install pandas-profiling

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

print('LOAD DURATION: ', datetime.now() - start_time) # load time about 30 seconds

LOAD DURATION:  0:00:00.731530


In [2]:
%%time
property_data = pd.read_csv('NY property data.csv.zip', compression="zip")

CPU times: user 4.33 s, sys: 444 ms, total: 4.77 s
Wall time: 4.77 s


## Exclusion

In [3]:
excluded_owners = [
    'PARKCHESTER PRESERVAT',
    'PARKS AND RECREATION',
    'DCAS',
    'HOUSING PRESERVATION',
    'CITY OF NEW YORK',
    'DEPT OF ENVIRONMENTAL',
    'BOARD OF EDUCATION',
    "NEW YORK CITY HOUSING",
    'CNY/NYCTA',
    'NYC HOUSING PARTNERSH',
    'DEPARTMENT OF BUSINES',
    'DEPT OF TRANSPORTATIO',
    'MTA/LIRR',
    'PARCKHESTER PRESERVAT',
    'MH RESIDENTIAL 1, LLC',
    'LINCOLN PLAZA ASSOCIA',
    'UNITED STATES OF AMER',
    'U S GOVERNMENT OWNRD',
    'THE CITY OF NEW YORK',
    'NYS URBAN DEVELOPMENT',
    'NYS DEPT OF ENVIRONME',
    'CULTURAL AFFAIRS',
    'DEPT OF GENERAL SERVI',
    'DEPT RE-CITY OF NY'
]

property_data = (
    property_data
    .query("`OWNER` not in @excluded_owners")
    .reset_index(drop=True)
)

## Data Cleaning: Fill in Missing Values

1. Make a list of the owner names to remove. Remove all records with those owners. Make sure you get all this right.
2. Fill in the missing ZIPs
3. Fill in missing FULLVAL, AVLAND, AVTOT
4. Fill in missing STORIES
5. Fill in LTFRONT, LTDEPTH, BLDFRONT, BLDDEPTH

### Fill in Missing Zip

In [4]:
missing_zips = np.where(pd.isnull(property_data['ZIP']))[0]
len(missing_zips)

21772

If the zip on both the record before and after the record with the missing zip are the same, replace by that zip code

In [5]:
for i in range(len(missing_zips)):
    if(property_data.loc[missing_zips[i]+1, 'ZIP'] == property_data.loc[missing_zips[i]-1,'ZIP']):
        property_data.loc[missing_zips[i],'ZIP'] = property_data.loc[missing_zips[i]-1, 'ZIP']

In [6]:
missing_zips = np.where(pd.isnull(property_data['ZIP']))[0]
len(missing_zips)

10245

For the rest of the missing zips, I simply replace missing zips with the zip from record above it

In [7]:
for i in range(len(missing_zips)):
    property_data.loc[missing_zips[i],'ZIP'] = property_data.loc[missing_zips[i]-1,'ZIP']

In [8]:
missing_zips = np.where(pd.isnull(property_data['ZIP']))[0]
len(missing_zips)

0

### Fill in Missing FULLVAL, AVLAND, AVTOT

Calculate means for AVTOT, AVLAND, FULLVAL by taxclass, avoiding records with zeros

In [9]:
property_data['FULLVAL'].replace('NaN',0)
temp = property_data[property_data['FULLVAL']!=0]
mean_fullval = temp.groupby('TAXCLASS')['FULLVAL'].mean()
print(mean_fullval)

TAXCLASS
1     5.700059e+05
1A    3.352942e+05
1B    3.738399e+05
1C    7.615359e+05
1D    2.233614e+07
2     8.005832e+05
2A    8.639066e+05
2B    1.254525e+06
2C    7.723493e+05
3     8.510005e+04
4     2.772747e+06
Name: FULLVAL, dtype: float64


In [10]:
property_data['AVLAND'].replace('NaN',0)
temp_avland = property_data[property_data['AVLAND']!=0]
mean_avland = temp_avland.groupby('TAXCLASS')['AVLAND'].mean()
print(mean_avland)

TAXCLASS
1      14901.860425
1A      2247.472465
1B      9185.394750
1C      8225.658898
1D    709303.793103
2      88471.102578
2A     31370.740197
2B     54607.394911
2C     25635.944099
3      28135.636364
4     444996.049088
Name: AVLAND, dtype: float64


In [11]:
property_data['AVTOT'].replace('NaN',0)
temp_avtot = property_data[property_data['AVTOT']!=0]
mean_avtot = temp_avtot.groupby('TAXCLASS')['AVTOT'].mean()
print(mean_avtot)

TAXCLASS
1     2.502549e+04
1A    1.437036e+04
1B    9.189080e+03
1C    2.898447e+04
1D    1.166866e+06
2     3.602638e+05
2A    7.963525e+04
2B    1.790961e+05
2C    1.170418e+05
3     3.829505e+04
4     1.294237e+06
Name: AVTOT, dtype: float64


Substitute decent values for AVTOT, AVLAND, FULLVAL from averages by taxclass

In [12]:
%%time
for index in mean_fullval.index:
    property_data.loc[(property_data['FULLVAL']==0)&(property_data['TAXCLASS']==index),'FULLVAL']=mean_fullval[index]
    property_data.loc[(property_data['AVLAND']==0)&(property_data['TAXCLASS']==index),'AVLAND']=mean_avland[index]
    property_data.loc[(property_data['AVTOT']==0)&(property_data['TAXCLASS']==index),'AVTOT']=mean_avtot[index]

CPU times: user 4.48 s, sys: 5.64 ms, total: 4.48 s
Wall time: 4.36 s


### Fill in Missing STORIES

In [13]:
temp = property_data[property_data['STORIES'].isnull()]
len(temp)

43968

In addition, there should not be a value zero for this field, so we treat zeros as missing values as well.
I'm grouping by TAXCLASS, since that should be a pretty good description of the nature of the buildings

In [14]:
temp['TAXCLASS'].value_counts()

1B    19055
4     17979
2      3434
3      2416
1       877
2C      138
2B       34
2A       30
1A        5
Name: TAXCLASS, dtype: int64

In [15]:
mean_stories = property_data.groupby('TAXCLASS')['STORIES'].mean()
print(mean_stories)

TAXCLASS
1      2.111633
1A     1.656867
1B     4.000000
1C     3.052748
1D     1.068966
2     16.310549
2A     2.844032
2B     4.001635
2C     4.744977
3      1.000000
4      5.517257
Name: STORIES, dtype: float64


In [16]:
property_data.query("STORIES == 0").shape[0]

0

In [17]:
property_data['STORIES']=property_data['STORIES'].fillna(value=0)
for index in mean_stories.index:
    property_data.loc[(property_data['STORIES']==0)&(property_data['TAXCLASS']==index),'STORIES']=mean_stories[index]

In [18]:
property_data.head().transpose()

,0,1,2,3,4
RECORD,1,2,9,10,11
BBLE,1000010101,1000010201,1000041001,1000041002,1000041003
B,1,1,1,1,1
BLOCK,1,1,4,4,4
LOT,101,201,1001,1002,1003
EASEMENT,NaN,NaN,NaN,NaN,NaN
OWNER,U S GOVT LAND & BLDGS,U S GOVT LAND & BLDGS,"TRZ HOLDINGS, LLC","TRZ HOLDINGS, LLC","TRZ HOLDINGS, LLC"
BLDGCL,P7,Z9,R5,R5,R5
TAXCLASS,4,4,4,4,4
LTFRONT,500,27,0,0,0


### Fill in Missing LOT and Building Sizes

In [19]:
# since these 4 values do not have NAs, we need to replace 0s
# calculate groupwise average. First replace the 0's and 1's by NAs so they are not counted in calculating mean
#also treat the sizes of 1s as missing
property_data.loc[property_data['LTFRONT']==0,'LTFRONT']=np.nan
property_data.loc[property_data['LTDEPTH']==0,'LTDEPTH']=np.nan
property_data.loc[property_data['BLDFRONT']==0,'BLDFRONT']=np.nan
property_data.loc[property_data['BLDDEPTH']==0,'BLDDEPTH']=np.nan
property_data.loc[property_data['LTFRONT']==1,'LTFRONT']=np.nan
property_data.loc[property_data['LTDEPTH']==1,'LTDEPTH']=np.nan
property_data.loc[property_data['BLDFRONT']==1,'BLDFRONT']=np.nan
property_data.loc[property_data['BLDDEPTH']==1,'BLDDEPTH']=np.nan

In [20]:
# calculate the mean (mean function ignores NAs but not 0s hence we converted 0 to NA)
mean_LTFRONT = property_data.groupby(property_data['TAXCLASS'])['LTFRONT'].mean()
mean_LTDEPTH = property_data.groupby(property_data['TAXCLASS'])['LTDEPTH'].mean()
mean_BLDFRONT = property_data.groupby(property_data['TAXCLASS'])['BLDFRONT'].mean()
mean_BLDDEPTH = property_data.groupby(property_data['TAXCLASS'])['BLDDEPTH'].mean()

In [21]:
# impute values
for index in mean_LTFRONT.index:
    property_data.loc[(property_data['LTFRONT'].isnull())&(property_data['TAXCLASS']==index),'LTFRONT']=mean_LTFRONT[index]
    property_data.loc[(property_data['LTDEPTH'].isnull())&(property_data['TAXCLASS']==index),'LTDEPTH']=mean_LTDEPTH[index]
    property_data.loc[(property_data['BLDFRONT'].isnull())&(property_data['TAXCLASS']==index),'BLDFRONT']=mean_BLDFRONT[index]
    property_data.loc[(property_data['BLDDEPTH'].isnull())&(property_data['TAXCLASS']==index),'BLDDEPTH']=mean_BLDDEPTH[index]

## Feature Engineering: Add New Variables
S1 = LTFRONT * LTDEPTH
S2 = BLDFRONT * BLDDEPTH
S3 = S2 * STORIES

In [22]:
# convert ZIP from float to integers
# convert ZIP from float to integers
property_data['ZIP']  = property_data['ZIP'].astype(str)
property_data['zip3'] = property_data['ZIP'].str[:3]

In [23]:
property_data['ltsize'] = property_data['LTFRONT'] * property_data['LTDEPTH']
property_data['bldsize'] = property_data['BLDFRONT'] * property_data['BLDDEPTH']
property_data['bldvol'] = property_data['bldsize'] * property_data['STORIES']

In [24]:
# make variables that are typical measures of how normal a property is
property_data['r1'] = property_data['FULLVAL'] / property_data['ltsize']
property_data['r2'] = property_data['FULLVAL'] / property_data['bldsize']
property_data['r3'] = property_data['FULLVAL'] / property_data['bldvol']
property_data['r4'] = property_data['AVLAND'] / property_data['ltsize']
property_data['r5'] = property_data['AVLAND'] / property_data['bldsize']
property_data['r6'] = property_data['AVLAND'] / property_data['bldvol']
property_data['r7'] = property_data['AVTOT'] / property_data['ltsize']
property_data['r8'] = property_data['AVTOT'] / property_data['bldsize']
property_data['r9'] = property_data['AVTOT'] / property_data['bldvol']

In [25]:
ninevars = ['r1','r2','r3','r4','r5','r6','r7','r8','r9']
zip5_mean = property_data.groupby('ZIP')[ninevars].mean()
zip3_mean = property_data.groupby('zip3')[ninevars].mean()
taxclass_mean = property_data.groupby('TAXCLASS')[ninevars].mean()
borough_mean = property_data.groupby('B')[ninevars].mean()

In [26]:
property_data = property_data.join(zip5_mean, on='ZIP', rsuffix='_zip5')
property_data = property_data.join(zip3_mean, on='zip3', rsuffix='_zip3')
property_data = property_data.join(taxclass_mean, on='TAXCLASS', rsuffix='_taxclass')
property_data = property_data.join(borough_mean, on='B', rsuffix='_boro')

In [27]:
rsuffix = ['_zip5','_zip3','_taxclass','_boro']
for var in ninevars:
    for r in rsuffix:
        property_data[str(var)+r] = property_data[var] / property_data[str(var)+r]

In [28]:
property_data.to_pickle("property_preprocessed.pickle")

In [29]:
cols = ['r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8', 'r9',
       'r1_zip5', 'r2_zip5', 'r3_zip5', 'r4_zip5', 'r5_zip5', 'r6_zip5',
       'r7_zip5', 'r8_zip5', 'r9_zip5', 'r1_zip3', 'r2_zip3', 'r3_zip3',
       'r4_zip3', 'r5_zip3', 'r6_zip3', 'r7_zip3', 'r8_zip3', 'r9_zip3',
       'r1_taxclass', 'r2_taxclass', 'r3_taxclass', 'r4_taxclass',
       'r5_taxclass', 'r6_taxclass', 'r7_taxclass', 'r8_taxclass',
       'r9_taxclass', 'r1_boro', 'r2_boro', 'r3_boro', 'r4_boro', 'r5_boro',
       'r6_boro', 'r7_boro', 'r8_boro', 'r9_boro']

{}
property_data[cols].describe().round(2).transpose().to_csv('stats_on_vars.csv')

In [30]:
property_data[cols].describe().round(2).transpose()

,count,mean,std,min,25%,50%,75%,max
r1,1046826.0,213.57,532.56,0.0,76.26,152.86,243.18,138637.34
r2,1046826.0,536.42,1027.44,0.0,211.82,500.00,683.82,310551.57
r3,1046826.0,244.08,498.82,0.0,82.50,239.06,336.23,227500.00
r4,1046826.0,10.46,69.98,0.0,2.35,4.60,7.23,22249.80
r5,1046826.0,22.82,378.51,0.0,6.20,14.98,20.29,334636.37
r6,1046826.0,10.04,178.33,0.0,2.02,7.15,10.04,111545.46
r7,1046826.0,26.83,203.87,0.0,5.28,8.58,13.53,64711.85
r8,1046826.0,50.46,932.20,0.0,17.47,26.98,36.00,871362.22
r9,1046826.0,19.07,327.02,0.0,7.09,12.68,17.58,290454.07
r1_zip5,1046826.0,1.00,2.16,0.0,0.46,0.92,1.23,747.76
